In [1]:
import pandas as pd

BASE_PATH = "C:\\Users\\Haroldo Duraes\\Desktop\\GOvGO\\v0\\#DATA\\PNCP\\"
CAT_PATH = BASE_PATH + "CAT\\"
FILE = CAT_PATH + "Lista_CATSER_Corrigida_Acentuada.xlsx"
ABA = "SERVIÇOS"

In [4]:
df = pd.read_excel(FILE, sheet_name=ABA)
display(df.head())

,Tipo Material Serviço,CodGrupo,Grupo,CodClasse,Classe,CodigoServico,Serviço,Sit Atual Mat Serv
0,Serviço,'-9,NAO SE APLICA,'-7,INVALIDO,15377,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,Ativo
1,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25852,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,Ativo
2,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25860,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,Ativo
3,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25879,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,Ativo
4,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25887,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,Ativo


In [7]:
# Primeiro, tratar os casos de "INVALIDO" na coluna "Classe"
print(f"Número de registros com Classe INVALIDO antes da correção: {(df['Classe'] == 'INVALIDO').sum()}")

# Criar cópias para não modificar os dados durante a iteração
df_corrigido = df.copy()

# Substituir onde Classe é INVALIDO
mask_invalido = df_corrigido['Classe'] == 'INVALIDO'
df_corrigido.loc[mask_invalido, 'Classe'] = df_corrigido.loc[mask_invalido, 'Grupo']
df_corrigido.loc[mask_invalido, 'CodClasse'] = df_corrigido.loc[mask_invalido, 'CodGrupo']

print(f"Número de registros com Classe INVALIDO após correção: {(df_corrigido['Classe'] == 'INVALIDO').sum()}")

# Criar um novo DataFrame para o CATSER com as colunas desejadas
catser_df = pd.DataFrame()

# Copiar as colunas simples
catser_df['CODITEM'] = df_corrigido['CodigoServico']
catser_df['ITEM'] = df_corrigido['Serviço']

# Concatenar os códigos com suas descrições
catser_df['GRUPO'] = df_corrigido['CodGrupo'].astype(str) + ' - ' + df_corrigido['Grupo']
catser_df['CLASSE'] = df_corrigido['CodClasse'].astype(str) + ' - ' + df_corrigido['Classe']

# Exibir as primeiras linhas do DataFrame resultante
print("\nDataFrame CATSER com as colunas solicitadas:")
display(catser_df.head())

# Informações sobre o DataFrame
print(f"\nDimensões do DataFrame: {catser_df.shape}")
print(f"Número de serviços únicos: {catser_df['ITEM'].nunique()}")

# Verificar se ainda há inconsistências (classes ou grupos vazios)
print(f"\nVerificação de consistência:")
print(f"Número de registros com CLASSE contendo 'INVALIDO': {catser_df['CLASSE'].str.contains('INVALIDO').sum()}")

Número de registros com Classe INVALIDO antes da correção: 688
Número de registros com Classe INVALIDO após correção: 0

DataFrame CATSER com as colunas solicitadas:


,CODITEM,ITEM,GRUPO,CLASSE
0,15377,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,'-9 - NAO SE APLICA,'-9 - NAO SE APLICA
1,25852,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...
2,25860,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...
3,25879,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...
4,25887,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...



Dimensões do DataFrame: (2823, 4)
Número de serviços únicos: 2820

Verificação de consistência:
Número de registros com CLASSE contendo 'INVALIDO': 0


In [10]:
# Adicionar a coluna CATEGORIA ao catser_df
catser_df['CATEGORIA'] = catser_df['GRUPO'] + '; ' + catser_df['CLASSE']

# Definir o número máximo de itens por CATEGORIA
n = 10  # Você pode ajustar este valor conforme necessário

# Criar o sample_df com no máximo n itens por CATEGORIA
def sample_category(group):
    if len(group) <= n:
        return group
    else:
        return group.sample(n, random_state=42)  # Usando random_state para reprodutibilidade

# Aplicar a função de amostragem por CATEGORIA
sample_df = catser_df.groupby('CATEGORIA').apply(sample_category).reset_index(drop=True)

# Exibir informações sobre os DataFrames
print(f"DataFrame original: {catser_df.shape[0]} itens")
print(f"DataFrame amostrado: {sample_df.shape[0]} itens")
print(f"Número de categorias: {sample_df['CATEGORIA'].nunique()}")

# Mostrar as primeiras linhas do sample_df
print("\nPrimeiras linhas do sample_df:")
display(sample_df.head())

# Verificar a distribuição dos itens por categoria
categoria_counts = sample_df.groupby('CATEGORIA').size().sort_values(ascending=False)
print("\nDistribuição de itens por categoria (top 10):")
display(categoria_counts.head(10))

# Verificar categorias com apenas um item
single_item_categories = categoria_counts[categoria_counts == 1]
print(f"\nNúmero de categorias com apenas um item: {len(single_item_categories)}")

DataFrame original: 2823 itens
DataFrame amostrado: 1032 itens
Número de categorias: 271

Primeiras linhas do sample_df:


C:\Users\Haroldo Duraes\AppData\Local\Temp\ipykernel_37824\2170817454.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = catser_df.groupby('CATEGORIA').apply(sample_category).reset_index(drop=True)


,CODITEM,ITEM,GRUPO,CLASSE,CATEGORIA
0,15377,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,'-9 - NAO SE APLICA,'-9 - NAO SE APLICA,'-9 - NAO SE APLICA; '-9 - NAO SE APLICA
1,25852,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
2,25860,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
3,25879,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
4,25887,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...,1111 - SERVIÇOS DE DESENVOLVIMENTO DE SOFTWAR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...



Distribuição de itens por categoria (top 10):


CATEGORIA
711 - SERVIÇOS DE INTERMEDIAÇÃO FINANCEIRA, EXCETO SERVIÇOS BANCÁRIO DE INVESTIMENTO, SERVIÇOS DE SEGUROS E DE PENSÕES; 711 - SERVIÇOS DE INTERMEDIAÇÃO FINANCEIRA, EXCETO SERVIÇOS BANCÁRIO DE INVESTIMENTO, SERVIÇOS DE SEGUROS E DE PENSÕES    10
871 - SERVIÇOS DE MANUTENÇÃO E REPARO DE PRODUTOS FABRICADOS DE METAL,MAQUINARIA E EQUIPAMENTOS; 871 - SERVIÇOS DE MANUTENÇÃO E REPARO DE PRODUTOS FABRICADOS DE METAL,MAQUINARIA E EQUIPAMENTOS                                                  10
835 - SERVIÇOS CIENTÍFICOS E OUTROS SERVIÇOS TÉCNICOS; 835 - SERVIÇOS CIENTÍFICOS E OUTROS SERVIÇOS TÉCNICOS                                                                                                                                      10
931 - SERVIÇOS DE SAÚDE HUMANA; 9311 - SERVIÇOS HOSPITALARES                                                                                                                                                                                      10
931 - SERV


Número de categorias com apenas um item: 94


In [11]:
# Contar quantos itens existem para cada CATEGORIA
counts = sample_df.groupby('CATEGORIA').size()

# Criar uma coluna que identifica se a CATEGORIA tem apenas um item
sample_df['single_item_category'] = sample_df['CATEGORIA'].apply(lambda x: counts[x] == 1)

# Inicializar os dataframes de treinamento e validação
training_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])
validation_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])

# Função para separar os itens em treinamento e validação
def split_category(group):
    global training_df, validation_df
    
    category = group['CATEGORIA'].iloc[0]
    
    # Se a categoria tiver apenas um item
    if group['single_item_category'].iloc[0]:
        # Adicionar o item tanto em treinamento quanto em validação
        item_data = pd.DataFrame({
            'ITEM': [group['ITEM'].iloc[0]],
            'CATEGORIA': [category]
        })
        training_df = pd.concat([training_df, item_data], ignore_index=True)
        validation_df = pd.concat([validation_df, item_data], ignore_index=True)
    else:
        # Pegar 1 item para treinamento
        training_item = group.iloc[[0]]
        training_data = pd.DataFrame({
            'ITEM': [training_item['ITEM'].iloc[0]],
            'CATEGORIA': [category]
        })
        training_df = pd.concat([training_df, training_data], ignore_index=True)
        
        # Pegar o restante para validação
        validation_items = group.iloc[1:]
        validation_data = pd.DataFrame({
            'ITEM': validation_items['ITEM'].tolist(),
            'CATEGORIA': validation_items['CATEGORIA'].tolist()
        })
        validation_df = pd.concat([validation_df, validation_data], ignore_index=True)

# Aplicar a divisão para cada CATEGORIA
sample_df.groupby('CATEGORIA').apply(split_category)

# Exibir informações sobre os conjuntos
print(f"Conjunto de Treinamento: {training_df.shape[0]} itens")
print(f"Conjunto de Validação: {validation_df.shape[0]} itens")

# Mostrar exemplos de ambos os conjuntos
print("\nExemplos do conjunto de treinamento:")
display(training_df.head())

print("\nExemplos do conjunto de validação:")
display(validation_df.head())

# Verificar número de categorias em cada conjunto
print(f"\nNúmero de categorias em treinamento: {training_df['CATEGORIA'].nunique()}")
print(f"Número de categorias em validação: {validation_df['CATEGORIA'].nunique()}")

# Verificar se todas as categorias com um único item estão em ambos os conjuntos
single_cats = counts[counts == 1].index.tolist()
training_single = [cat in training_df['CATEGORIA'].values for cat in single_cats]
validation_single = [cat in validation_df['CATEGORIA'].values for cat in single_cats]

print(f"\nCategorias de item único presentes no treinamento: {sum(training_single)}/{len(single_cats)}")
print(f"Categorias de item único presentes na validação: {sum(validation_single)}/{len(single_cats)}")

Conjunto de Treinamento: 271 itens
Conjunto de Validação: 855 itens

Exemplos do conjunto de treinamento:


C:\Users\Haroldo Duraes\AppData\Local\Temp\ipykernel_37824\3987878489.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df.groupby('CATEGORIA').apply(split_category)


,ITEM,CATEGORIA
0,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,'-9 - NAO SE APLICA; '-9 - NAO SE APLICA
1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
2,MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO D...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
3,"MANUTENCAO DE SOFTWARE (CORRETIVA, PREVENTIVA,...",112 - SERVIÇOS DE MANUTENÇÃO E SUSTENTAÇÃO DE ...
4,DOCUMENTACAO DE SOFTWARE,113 - SERVIÇOS DE DOCUMENTAÇÃO DE SOFTWARE; 11...



Exemplos do conjunto de validação:


,ITEM,CATEGORIA
0,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,'-9 - NAO SE APLICA; '-9 - NAO SE APLICA
1,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
2,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
3,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
4,DESENVOLVIMENTO E/OU EVOLUCAO DE SOFTWATE - DI...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...



Número de categorias em treinamento: 271
Número de categorias em validação: 271

Categorias de item único presentes no treinamento: 94/94
Categorias de item único presentes na validação: 94/94


In [14]:
training_df.shape, validation_df.shape

((271, 2), (855, 2))

In [19]:
# Adicionar a palavra "SERVIÇO" no início de cada categoria
training_df['CATEGORIA'] = "SERVIÇO; " + training_df['CATEGORIA']
validation_df['CATEGORIA'] = "SERVIÇO; " + validation_df['CATEGORIA']

# Mostrar exemplos de ambos os conjuntos
print("\nExemplos do conjunto de treinamento:")
display(training_df.head())

print("\nExemplos do conjunto de validação:")
display(validation_df.head())


Exemplos do conjunto de treinamento:


,ITEM,CATEGORIA
0,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,SERVIÇO; '-9 - NAO SE APLICA; '-9 - NAO SE APLICA
1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
2,MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO D...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
3,"MANUTENCAO DE SOFTWARE (CORRETIVA, PREVENTIVA,...",SERVIÇO; 112 - SERVIÇOS DE MANUTENÇÃO E SUSTEN...
4,DOCUMENTACAO DE SOFTWARE,SERVIÇO; 113 - SERVIÇOS DE DOCUMENTAÇÃO DE SOF...



Exemplos do conjunto de validação:


,ITEM,CATEGORIA
0,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,SERVIÇO; '-9 - NAO SE APLICA; '-9 - NAO SE APLICA
1,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
2,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
3,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
4,DESENVOLVIMENTO E/OU EVOLUCAO DE SOFTWATE - DI...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...


In [20]:
# Definir o caminho para salvar o arquivo Excel
output_path = f"{CAT_PATH}catser_datasets.xlsx"

# Criar um ExcelWriter para salvar múltiplas abas em um único arquivo Excel
with pd.ExcelWriter(output_path) as writer:
    # Salvar cada DataFrame em uma aba separada
    training_df.to_excel(writer, sheet_name="training", index=False)
    validation_df.to_excel(writer, sheet_name="validation", index=False)
    sample_df.to_excel(writer, sheet_name="sample", index=False)
    


print(f"Datasets salvos em: {output_path}")
print("Abas salvas:")
print("- training: conjunto de treinamento com prefixo SERVIÇO")
print("- validation: conjunto de validação com prefixo SERVIÇO")
print("- sample: dataset reduzido por amostragem")

Datasets salvos em: C:\Users\Haroldo Duraes\Desktop\GOvGO\v0\#DATA\PNCP\CAT\catser_datasets.xlsx
Abas salvas:
- training: conjunto de treinamento com prefixo SERVIÇO
- validation: conjunto de validação com prefixo SERVIÇO
- sample: dataset reduzido por amostragem
